In [1]:
# %load blackbox_fwdsolve_grad.py
import pymc3 as pm
import numpy as np
import theano.tensor as tt
import matplotlib
import matplotlib.pyplot as plt
import arviz as az
import scipy.io as spio
import warnings
import matlab.engine

In [2]:
engine = matlab.engine.start_matlab()

# matlab function that returns the [jacobian, homog_voltages, voltages] of a EIDORS model of a2C with a user specificed number of electrodes
stuff = engine.calc_j_hvoltages(nargout=3)
jacobian = np.asarray(stuff[0])
homog_voltages = np.asarray(stuff[1])
data = np.asarray(stuff[2])
engine.quit

class LogLikeWithGrad(tt.Op): # ????

    itypes = [tt.dvector]  # expects a vector of parameter values when called
    otypes = [tt.dscalar]  # outputs a single scalar value (the log likelihood)

    def __init__(self, loglike, jacobian, data, homog_voltages):

        #Initialise the Op with various things that our log-likelihood function requires.

        # add inputs as class attributes
        self.likelihood = loglike
        self.data = data
        #self.sigma = sigma
        self.jacobian = jacobian
        self.homog_voltages = homog_voltages

        # initialise the gradient Op (below)
        self.logpgrad = LogLikeGrad(self.likelihood, self.jacobian, self.data, self.homog_voltages)

    def perform(self, node, inputs, outputs):
        # the method that is used when calling the Op
        (parameters,) = inputs  # this will contain my variables ???
        # call the log-likelihood function
        logl = self.likelihood(parameters, self.data, self.jacobian, self.homog_voltages)

        outputs[0][0] = np.array(logl)  # output the log-likelihood

    def grad(self, inputs, g):
        # the method that calculates the gradients - it actually returns the
        # vector-Jacobian product - g[0] is a vector of parameter values
        (parameters,) = inputs  # our parameters
        return [g[0] * self.logpgrad(parameters)]

In [3]:
class LogLikeGrad(tt.Op):
    """
    This Op will be called with a vector of values and also return a vector of
    values - the gradients in each dimension.
    """

    itypes = [tt.dvector]
    otypes = [tt.dvector]

    def __init__(self, loglike, jacobian, data, homog_voltages):
        """
        Initialise with various things that the function requires. Below
        are the things that are needed in this particular example.
        Parameters
        ----------
        loglike:
            The log-likelihood (or whatever) function we've defined
        data:
            The "observed" data that our log-likelihood function takes in
        x:
            The dependent variable (aka 'x') that our model requires
        sigma:
            The noise standard deviation that out function requires.
        """

        # add inputs as class attributes
        self.likelihood = loglike
        self.data = data
        #self.sigma = sigma
        self.jacobian = jacobian
        self.homog_voltages = homog_voltages

    def perform(self, node, inputs, outputs):
        (parameters,) = inputs
        # define version of likelihood function to pass to derivative function
        def lnlike(values):
            return self.likelihood(values, self.data, self.jacobian, self.homog_voltages)
        # calculate gradients
        grads = gradients(parameters, lnlike)

        outputs[0][0] = grads

In [4]:
def my_model(conductivity, jacobian, homog_voltages):
    # Forward Solver: homog_voltages + jacobian(conductivity - ones)
    diff = np.subtract(conductivity, np.ones(64))
    j = np.multiply(jacobian, diff)
    voltages = np.add(homog_voltages, j)

    # return the numpy array
    return voltages

def gradients(vals, func, releps=1e-3, abseps=None, mineps=1e-9, reltol=1e-3,
              epsscale=0.5):
    """
    Calculate the partial derivatives of a function at a set of values. The
    derivatives are calculated using the central difference, using an iterative
    method to check that the values converge as step size decreases.
    Parameters
    ----------
    vals: array_like
        A set of values, that are passed to a function, at which to calculate
        the gradient of that function
    func:
        A function that takes in an array of values.
    releps: float, array_like, 1e-3
        The initial relative step size for calculating the derivative.
    abseps: float, array_like, None
        The initial absolute step size for calculating the derivative.
        This overrides `releps` if set.
        `releps` is set then that is used.
    mineps: float, 1e-9
        The minimum relative step size at which to stop iterations if no
        convergence is achieved.
    epsscale: float, 0.5
        The factor by which releps if scaled in each iteration.
    Returns
    -------
    grads: array_like
        An array of gradients for each non-fixed value.
    """

    grads = np.zeros(len(vals))

    # maximum number of times the gradient can change sign
    flipflopmax = 10.

    # set steps
    if abseps is None:
        if isinstance(releps, float):
            eps = np.abs(vals)*releps
            eps[eps == 0.] = releps  # if any values are zero set eps to releps
            teps = releps*np.ones(len(vals))
        elif isinstance(releps, (list, np.ndarray)):
            if len(releps) != len(vals):
                raise ValueError("Problem with input relative step sizes")
            eps = np.multiply(np.abs(vals), releps)
            eps[eps == 0.] = np.array(releps)[eps == 0.]
            teps = releps
        else:
            raise RuntimeError("Relative step sizes are not a recognised type!")
    else:
        if isinstance(abseps, float):
            eps = abseps*np.ones(len(vals))
        elif isinstance(abseps, (list, np.ndarray)):
            if len(abseps) != len(vals):
                raise ValueError("Problem with input absolute step sizes")
            eps = np.array(abseps)
        else:
            raise RuntimeError("Absolute step sizes are not a recognised type!")
        teps = eps

    # for each value in vals calculate the gradient
    count = 0
    for i in range(len(vals)):
        # initial parameter diffs
        leps = eps[i]
        cureps = teps[i]

        flipflop = 0

        # get central finite difference
        fvals = np.copy(vals)
        bvals = np.copy(vals)

        # central difference
        fvals[i] += 0.5*leps  # change forwards distance to half eps
        bvals[i] -= 0.5*leps  # change backwards distance to half eps
        cdiff = (func(fvals)-func(bvals))/leps

        while 1:
            fvals[i] -= 0.5*leps  # remove old step
            bvals[i] += 0.5*leps

            # change the difference by a factor of two
            cureps *= epsscale
            if cureps < mineps or flipflop > flipflopmax:
                # if no convergence set flat derivative (TODO: check if there is a better thing to do instead)
                warnings.warn("Derivative calculation did not converge: setting flat derivative.")
                grads[count] = 0.
                break
            leps *= epsscale

            # central difference
            fvals[i] += 0.5*leps  # change forwards distance to half eps
            bvals[i] -= 0.5*leps  # change backwards distance to half eps
            cdiffnew = (func(fvals)-func(bvals))/leps

            if cdiffnew == cdiff:
                grads[count] = cdiff
                break

            # check whether previous diff and current diff are the same within reltol
            rat = (cdiff/cdiffnew)
            if np.isfinite(rat) and rat > 0.:
                # gradient has not changed sign
                if np.abs(1.-rat) < reltol:
                    grads[count] = cdiffnew
                    break
                else:
                    cdiff = cdiffnew
                    continue
            else:
                cdiff = cdiffnew
                flipflop += 1
                continue

        count += 1

    return grads



covar = spio.loadmat('/Users/maxlewter/Desktop/Code/covarmatrix.mat', squeeze_me=True)
covar2 = covar['covar']

# define your really-complicated likelihood function that uses loads of external codes
def my_loglike(parameters, data, jacobian, homog_voltages):
    conductivity = parameters[:-1]
    #print(conductivity)
    lognoisevariance = parameters[-1]
    #print('lognoisevariance is', lognoisevariance)
    sigma = np.sqrt(np.exp(lognoisevariance))
    v_covar = np.square(sigma)*np.eye(64)
    mu = my_model(conductivity, jacobian, homog_voltages)
    sigma = np.sqrt(np.exp(lognoisevariance))
    #print("sigma is", sigma)
    z = np.subtract(data, mu) / sigma
    z = np.matrix(z)
    loglik = -.5*np.log(np.linalg.det(v_covar)) - (64/2) * np.log(2 * 3.14159) - .5 * z.H * z #log likelihood equation
    loglik = loglik.item(0)
    #print(loglik)
    return loglik


# create our Op
logl = LogLikeWithGrad(my_loglike, jacobian, data, homog_voltages)
basic_model = pm.Model()
# use PyMC3 to sample from log-likelihood
with basic_model:
    mu = np.ones(64, dtype=int)
    #Took out covariance to make sigma a set number
    cond = pm.MvNormal("cond", mu=mu, cov=covar2, shape=64) #correct covar data structure?
    #cond = pm.Normal("cond", mu=1, sigma=1.2, shape=64)
    lognoisevariance = pm.Normal("lognoisevariance", mu=0, sigma=1, shape=1)

    # convert conductivity to a tensor vector
    condt = tt.as_tensor_variable(cond)
    lognvt = tt.as_tensor_variable(lognoisevariance)
    parameters = tt.concatenate([condt, lognvt])

    # use a DensityDist (use a lamdba function to "call" the Op) ????
    #pymc3 densitydist has issues, using pm.Potential because this is supposed to work 
    #pm.DensityDist("likelihood", lambda v: logl(v), observed={"v": parameters})
    pm.Potential("likelihood", logl(parameters))
    map_estimate = pm.find_MAP(model=basic_model)
    print(map_estimate)
    trace = pm.sample(5000, tune= 1000, return_inferencedata=False, idata_kwargs={"density_dist_obs": False}) #draw samples from posterior
    az.plot_trace(trace);
    plt.show();



<ipython-input-4-a623bb3c19af>:147: RuntimeWarning: divide by zero encountered in log
  loglik = -.5*np.log(np.linalg.det(v_covar)) - (64/2) * np.log(2 * 3.14159) - .5 * z.H * z #log likelihood equation
<ipython-input-4-a623bb3c19af>:96: UserWarning: Derivative calculation did not converge: setting flat derivative.
  warnings.warn("Derivative calculation did not converge: setting flat derivative.")


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


{'cond': array([1.02087141, 1.07437325, 0.94155623, 1.07437325, 1.05632788,
       0.96446174, 1.02212861, 0.96446174, 0.98123427, 0.98123427,
       1.00763056, 1.01596597, 0.99324338, 0.99324338, 1.01596597,
       1.00763056, 0.99741063, 0.99741063, 1.0030859 , 0.99936778,
       0.99922823, 0.99922823, 0.99936778, 1.0030859 , 1.00725272,
       0.9994347 , 1.00725272, 0.99605581, 1.00058421, 0.99479539,
       1.00291614, 0.99958152, 1.00291614, 0.99479539, 1.00058421,
       0.99605581, 0.99791138, 1.00073101, 0.99791138, 1.00042573,
       0.99953565, 1.00238274, 0.9991345 , 1.00029132, 0.9991345 ,
       1.00238274, 0.99953565, 1.00042573, 0.99969033, 1.00046276,
       1.00046276, 0.99969033, 1.00060945, 0.99979333, 0.99959677,
       0.99963197, 0.99984307, 1.00019285, 1.00019285, 0.99984307,
       0.99963197, 0.99959677, 0.99979333, 1.00060945]), 'lognoisevariance': array([-16.49944096])}


Multiprocess sampling (3 chains in 3 jobs)
NUTS: [lognoisevariance, cond]


RuntimeError: Chain 0 failed.

In [ ]:
#%whos
#print(map_estimate)

In [ ]:

# with basic_model:
#     # instantiate sampler
#     step = pm.Slice()

#     # draw 5000 posterior samples
#     trace = pm.sample(10, step=step, return_inferencedata=False, idata_kwargs={"density_dist_obs":False})
#     az.plot_trace(trace);
# with basic_model:
#     trace = pm.sample(10, return_inferencedata=True, idata_kwargs={"density_dist_obs": False}) #draw samples from posterior
#     az.plot_trace(trace);
#     plt.show();
# #     trace = pm.sample(10, return_inferencedata=False)
# #     az.plot_trace(trace)
# #     #display(az.summary(trace, roun_to=2))
